# 🌧️ Klasifikasi Curah Hujan Harian Menggunakan Metode Random Forest

## 📌 Deskripsi
Proyek ini bertujuan untuk mengklasifikasikan curah hujan harian berdasarkan berbagai variabel cuaca menggunakan metode **Random Forest**. Dengan memanfaatkan data cuaca dari **BMKG Bima** selama periode **2019-2023**, penelitian ini menguji berbagai pendekatan untuk meningkatkan performa model dalam mengklasifikasikan curah hujan ke dalam **empat kategori**.

## 📊 Dataset
Dataset yang digunakan mencakup 2.223 sampel data cuaca harian dengan 10 fitur utama:
- **Tanggal** ⏳
- **Temperatur Min, Max, dan Rata-rata 🌡️**
- **Kelembapan Rata-rata 💧**
- **Curah Hujan (Target) ☔**
- **Penyinaran Matahari ☀️**
- **Kecepatan dan Arah Angin 🌬️**

Distribusi kelas dalam dataset adalah sebagai berikut:
- **Kelas 1** (Tidak Hujan) → 1.573 sampel
- **Kelas 2** (Hujan Ringan) → 546 sampel
- **Kelas 3** (Hujan Sedang) → 92 sampel
- **Kelas 4** (Hujan Lebat) → 12 sampel

## 🛠️ Pendekatan
Untuk meningkatkan akurasi model, penelitian ini melakukan **tiga percobaan utama**:
1. **Klasifikasi Langsung**
   - Model Random Forest diterapkan langsung pada dataset asli.
2. **Penanganan Ketidakseimbangan Data**
   - Menggunakan **SMOTE** untuk mengatasi distribusi kelas yang tidak merata.
3. **Seleksi Fitur**
   - Hanya menggunakan fitur paling berpengaruh berdasarkan analisis **Feature Importance**.

## 🏆 Hasil Evaluasi

| Percobaan               | Macro F1-Score |
|-------------------------|---------------|
| Klasifikasi Langsung    | 0.8664        |
| SMOTE (Resampling)      | 1.0000        |
| Seleksi Fitur           | 1.0000        |


Dari hasil di atas, pendekatan **SMOTE dan seleksi fitur** menghasilkan peningkatan performa model secara signifikan.

## 🚀 Teknologi yang Digunakan
- **Python** 🐍
- **Scikit-Learn** 🤖
- **Pandas & NumPy** 📊
- **Imbalanced-learn (SMOTE)** ⚖️
- **Plotly (Visualisasi)** 📈

## 🔧 Cara Menggunakan
1. Clone repositori ini:
   ```bash
   git clone https://github.com/username/klasifikasi-curah-hujan-rf.git
   ```
2. Install dependensi:
   ```bash
   pip install -r requirements.txt
   ```
3. Jalankan notebook di Google Colab atau Jupyter Notebook.

## 🎯 Kesimpulan
Studi ini menunjukkan bahwa dengan teknik **resampling (SMOTE) dan seleksi fitur**, model **Random Forest** dapat memberikan hasil klasifikasi yang lebih akurat terhadap curah hujan harian. Penelitian ini dapat menjadi dasar untuk analisis lebih lanjut terkait prediksi curah hujan dan mitigasi bencana cuaca ekstrem. 🌦️

---
🚀 **Mari bersama membangun solusi berbasis data untuk memahami pola curah hujan!**

##🎯 **Persiapan Project**

In [ ]:
# @title 📥 Import Library

# Import library utama
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

# Library untuk preprocessing
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split

# Library untuk modeling dan evaluasi
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, f1_score

from prettytable import PrettyTable
from imblearn.over_sampling import SMOTE

In [ ]:
# @title 📂 Import Dataset {"display-mode":"form"}
data_cuaca = pd.read_excel('/content/drive/MyDrive/Portofolio Data Analyst/data-cuaca-harian.xlsx')
data_cuaca

,Tanggal,temp_min,temp_max,temp_avg,kelembapan_avg,curah_hujan,penyinaran_matahari,kec_angin,arah_angin,kec_angin_avg
0,1012019,25.4,32.0,26.8,90.0,2.5,5.5,3.0,350.0,0.0
1,2012019,24.8,32.0,27.0,89.0,32.6,1.6,4.0,340.0,1.0
2,3012019,25.4,31.8,26.0,94.0,20.5,1.7,3.0,10.0,0.0
3,4012019,24.0,31.0,26.6,88.0,4.0,0.0,6.0,10.0,1.0
4,5012019,24.2,33.0,26.9,87.0,0.3,4.6,4.0,10.0,1.0
...,...,...,...,...,...,...,...,...,...,...
2218,27012025,25.0,30.0,27.4,86.0,1.5,6.2,5.0,40.0,1.0
2219,28012025,25.4,31.6,27.8,87.0,8888.0,5.2,6.0,360.0,1.0
2220,29012025,25.2,31.4,27.1,87.0,0.0,3.5,5.0,360.0,1.0
2221,30012025,25.2,31.6,26.5,93.0,14.2,6.2,5.0,110.0,1.0


##📊 **Exploratory Data Analysis**

In [ ]:
# @title ℹ️ Informasi Dataset {"display-mode":"form"}
data_cuaca.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2223 entries, 0 to 2222
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Tanggal              2223 non-null   int64  
 1   temp_min             2178 non-null   float64
 2   temp_max             2201 non-null   float64
 3   temp_avg             2177 non-null   float64
 4   kelembapan_avg       2177 non-null   float64
 5   curah_hujan          2071 non-null   float64
 6   penyinaran_matahari  2199 non-null   float64
 7   kec_angin            2206 non-null   float64
 8   arah_angin           2205 non-null   float64
 9   kec_angin_avg        2206 non-null   float64
dtypes: float64(9), int64(1)
memory usage: 173.8 KB


**Analisis Informasi Awal Dataset**

Dataset ini terdiri dari 2223 baris dan 10 kolom dengan mayoritas tipe data numerik, namun kolom "Tanggal" masih dalam format integer yang perlu dikonversi ke datetime64 untuk analisis berbasis waktu. Beberapa kolom memiliki missing values, dengan yang tertinggi terdapat pada "curah_hujan" (6.84%), sementara lainnya di bawah 3%, sehingga perlu dilakukan penanganan seperti imputasi atau penghapusan baris jika terlalu banyak data yang hilang. Dari segi efisiensi, ukuran dataset masih kecil (173.8 KB) dan tidak ada masalah signifikan terkait tipe data selain pada kolom tanggal. Oleh karena itu, sebelum masuk ke tahap eksplorasi lebih lanjut, perlu dilakukan preprocessing untuk menangani missing values, konversi tipe data, serta pemeriksaan inkonsistensi atau outlier guna memastikan kualitas data sebelum digunakan dalam analisis atau pemodelan.

In [ ]:
# @title 📈 Statistik Deskriptif Dataset {"display-mode":"form"}
numerik = data_cuaca[['temp_min', 'temp_max', 'temp_avg', 'kelembapan_avg', 'curah_hujan', 'penyinaran_matahari', 'kec_angin', 'arah_angin', 'kec_angin_avg']]
numerik.describe()

,temp_min,temp_max,temp_avg,kelembapan_avg,curah_hujan,penyinaran_matahari,kec_angin,arah_angin,kec_angin_avg
count,2178.000000,2201.000000,2177.000000,2177.000000,2071.000000,2199.000000,2206.000000,2205.000000,2206.000000
mean,23.892654,33.193230,27.549150,80.874139,797.207774,6.636926,6.140073,181.380045,2.061197
std,1.574431,1.560604,1.224054,5.924134,2534.622266,2.817757,1.589894,100.334082,1.119754
min,18.000000,25.400000,23.100000,61.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,23.000000,32.200000,26.800000,77.000000,0.000000,4.800000,5.000000,150.000000,1.000000
50%,24.200000,33.200000,27.600000,81.000000,0.000000,7.400000,6.000000,180.000000,2.000000
75%,25.000000,34.200000,28.300000,85.000000,5.500000,8.700000,7.000000,200.000000,3.000000
max,27.200000,39.000000,31.400000,98.000000,8888.000000,11.700000,15.000000,360.000000,6.000000


**Analisis Statistik Deskriptif Dataset**
<br>Dari output statistik deskriptif yang diberikan, kita dapat menarik beberapa wawasan penting mengenai distribusi dan karakteristik setiap fitur dalam dataset.

  * Dataset mengandung missing values, terutama pada `curah_hujan`, `temp_avg`, dan `kelembapan_avg`.
  * Curah hujan memiliki distribusi yang sangat tidak seimbang, dengan sebagian besar hari tanpa hujan, tetapi ada outlier ekstrem (8888) yang perlu diperiksa lebih lanjut.
  * Temperatur dan kelembapan memiliki distribusi yang cukup stabil, dengan sedikit variasi.
  * Kecepatan angin cenderung rendah dalam banyak hari, tetapi bisa mencapai 15 m/s pada beberapa kesempatan.
  * Perlu penanganan lebih lanjut untuk outlier dan missing values sebelum analisis lebih lanjut atau pemodelan dilakukan.

In [ ]:
# @title 🔗 Korelasi Fitur Numerik {"display-mode":"form"}
cuaca = data_cuaca[['temp_avg', 'kelembapan_avg', 'curah_hujan', 'kec_angin_avg']]
colName = ['Temperatur', 'Kelembapan', 'Curah Hujan', 'Kec. Angin']
cuaca.columns = colName

corr = cuaca.corr()

# mask the upper triangle of the correlation matrix with NaN
mask = np.triu(np.ones_like(corr, dtype=bool))
corr = corr.mask(mask)

fig = px.imshow(corr, color_continuous_scale='purp',
                x=corr.index, y=corr.columns, text_auto=True,
                labels={"color":"R"}, title="Korelasi antar fitur numerik")

fig.update_traces(xgap=1, ygap=1)
fig.data[0].texttemplate = '%{z:.2f}'
fig.data[0].hovertemplate = 'x : %{x}<br>y : %{y}<br>R : %{z:.2f}<extra></extra>'

fig.update_layout(
    title={'x': 0.5,
           'xanchor': 'center',
           'font_family': 'Arial Black',
           'font_size': 20,
           },
    xaxis=dict(
              showline=True,
              linecolor='darkgrey'
          ),
    yaxis=dict(
              showline=True,
              linecolor='darkgrey'
          ),
    width=700,
    template='plotly_white',
    )

fig.add_annotation(xref='x domain', yref="y domain", x=0.5, y=1.13, font_size=14,
                   text="<i>Mayoritas fitur tidak menunjukkan hubungan linier yang signifikan</i>", showarrow=False)

fig.show()

**Analisis Korelasi Antar Fitur Numerik**
<br> Berdasarkan matriks korelasi yang diberikan, hubungan antara variabel dalam dataset dapat dianalisis sebagai berikut:

  * Korelasi antara fitur temperatur, kelembapan, curah hujan, dan kecepatan angin umumnya lemah, kecuali untuk hubungan negatif yang cukup kuat antara kelembapan dan kecepatan angin (-0.676).
  * Temperatur tidak memiliki pengaruh yang signifikan terhadap curah hujan atau kecepatan angin, tetapi sedikit berhubungan negatif dengan kelembapan.
  * Curah hujan tidak terlalu berkorelasi dengan kelembapan atau kecepatan angin, menunjukkan bahwa mungkin ada faktor lain yang lebih berpengaruh.
  * Hubungan negatif yang cukup kuat antara kelembapan dan kecepatan angin menunjukkan bahwa kondisi udara lembap sering kali diiringi dengan angin yang lebih tenang.


##📝 **Pra-Proses Data**

In [ ]:
# @title 🔄 Perubahan Format {"display-mode":"form"}
def isi_tanggal(tanggal):
    return tanggal.zfill(8)

# Mengubah format kolom 'Tanggal' menjadi int dan menambahkan 0 didepannya
data_cuaca['Tanggal'] = data_cuaca['Tanggal'].astype('str').apply(isi_tanggal)
data_cuaca['Tanggal'] = pd.to_datetime(data_cuaca['Tanggal'], format='%d%m%Y')

data_cuaca

,Tanggal,temp_min,temp_max,temp_avg,kelembapan_avg,curah_hujan,penyinaran_matahari,kec_angin,arah_angin,kec_angin_avg
0,2019-01-01,25.4,32.0,26.8,90.0,2.5,5.5,3.0,350.0,0.0
1,2019-01-02,24.8,32.0,27.0,89.0,32.6,1.6,4.0,340.0,1.0
2,2019-01-03,25.4,31.8,26.0,94.0,20.5,1.7,3.0,10.0,0.0
3,2019-01-04,24.0,31.0,26.6,88.0,4.0,0.0,6.0,10.0,1.0
4,2019-01-05,24.2,33.0,26.9,87.0,0.3,4.6,4.0,10.0,1.0
...,...,...,...,...,...,...,...,...,...,...
2218,2025-01-27,25.0,30.0,27.4,86.0,1.5,6.2,5.0,40.0,1.0
2219,2025-01-28,25.4,31.6,27.8,87.0,8888.0,5.2,6.0,360.0,1.0
2220,2025-01-29,25.2,31.4,27.1,87.0,0.0,3.5,5.0,360.0,1.0
2221,2025-01-30,25.2,31.6,26.5,93.0,14.2,6.2,5.0,110.0,1.0


**Analisis Konversi Kolom Tanggal dari Integer ke Datetime**
<br>Dalam dataset ini, kolom Tanggal awalnya disimpan dalam format integer `(misalnya: 01012019)`, kemudian dikonversi ke format datetime `(YYYY-MM-DD)`. Berikut adalah beberapa aspek penting dari perubahan ini:

  * Sebelumnya, Tanggal hanya berupa angka, sehingga sulit dibaca dan diproses sebagai data waktu.
  * Setelah dikonversi ke datetime, formatnya menjadi lebih intuitif `(YYYY-MM-DD)`, memudahkan analisis dan manipulasi data waktu.
  * Konversi dari integer ke datetime merupakan langkah yang sangat baik untuk meningkatkan keterbacaan dan kemudahan manipulasi data waktu.
  * Memungkinkan analisis berbasis waktu yang lebih mendalam, seperti tren musiman, perhitungan selisih hari, dan lainnya.

In [ ]:
# @title 🔬 Deteksi Outlier Setiap Fitur {"display-mode":"form"}
def deteksi_outlier(data, columns):
    outliers = {}
    for col in columns:
        Q1 = data[col].quantile(0.25)
        Q3 = data[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        outliers[col] = data[(data[col] < lower_bound) | (data[col] > upper_bound)]
    return outliers

outlier = deteksi_outlier(data_cuaca, numerik)

fig = go.Figure()
for col in numerik:
    fig.add_trace(go.Box(y=data_cuaca[col], name=col, boxpoints='outliers'))

fig.update_layout(
    title={'text': "Outlier Dari Setiap Fitur",
           'x': 0.5,
           'xanchor': 'center',
           'font_family': 'Arial Black',
           'font_size': 20,
           },
    xaxis=dict(
              showline=True,
              linecolor='darkgrey',
              title='Fitur'
          ),
    yaxis=dict(
              showline=True,
              linecolor='darkgrey',
              title='Nilai'
          ),
    width=700,
    showlegend=False
    )

fig.add_annotation(xref='x domain', yref="y domain", x=0.5, y=1.14, font_size=14,
                   text="<i>Terdapat satu kolom yang memiliki outlier</i>", showarrow=False)

fig.add_annotation(x=4, y=8888,
                  text="Outlier pada kolom curah_hujan",
                  showarrow=True,
                  arrowhead=2, ax=0, ay=50
                   )

fig.show()

**Analisis Outlier dari Setiap Fitur**
<br>Berdasarkan hasil pengamatan outlier yang menggunakan metode IQR (Interquartile Range). Didapatkanlah hasil sebagai berikut:

  * Variabel `curah_hujan` memiliki outlier ekstrem (nilai 8888)
  * Setelah ditelusuri, nilai 8888 ini merupakan simbol yang menandakan tidak adanya pengukuran pada hari tersebut. Keterangan ini bersumber dari BMKG itu sendiri.
  * Outlier pada variabel ini perlu dikonversi ke `NaN` sebelum analisis lebih lanjut.

  <br>

In [ ]:
# @title 🔧 Handling Outlier {"display-mode":"form"}
# Mengubah nilai 8888 menjadi NaN
data_cuaca['curah_hujan'] = data_cuaca['curah_hujan'].replace(8888, np.nan)

Seperti yang telah dijelaskan sebelumnya, karena outlier pada variabel `curah_hujan` merupakan placehokder yang menandakan tidak adanya pengukuran pada hari tersebut, maka untuk outlier tersebut diganti menjadi `NaN` yang selanjutnya akan diproses bersamaan dengan missing values lainnya.

In [ ]:
# @title 🔎 Deteksi Missing Values {"display-mode":"form"}

Missing_values = data_cuaca.isnull().sum().sort_values(ascending=False)
percent = (data_cuaca.isnull().sum()/data_cuaca.isnull().count()*100).sort_values(ascending=False)

missing_data = pd.concat([Missing_values, percent], axis=1, keys=['Missing values', 'Persentase'])
missing_data

,Missing values,Persentase
curah_hujan,337,15.159694
temp_avg,46,2.069276
kelembapan_avg,46,2.069276
temp_min,45,2.024291
penyinaran_matahari,24,1.079622
temp_max,22,0.989654
arah_angin,18,0.809717
kec_angin,17,0.764732
kec_angin_avg,17,0.764732
Tanggal,0,0.000000


**Analisis Missing Values pada Dataset**
<br>Berdasarkan hasil inspeksi, dataset memiliki missing values pada beberapa fitur numerik. Berikut adalah analisis dari jumlah nilai yang hilang di setiap kolom:

**Identifikasi Missing Values**
  * Kolom `curah_hujan` memiliki missing values terbanyak (337) → Ini adalah 15,16% dari total data (337/2223), yang cukup signifikan. Karena curah hujan adalah variabel utama dalam klasifikasi, perlu strategi penanganan yang tepat untuk menghindari bias dalam model.
  * Kolom `temp_avg` dan `kelembapan_avg` memiliki jumlah missing values yang sama (46) → Perlu diperiksa apakah ini terjadi karena perhitungan nilai rata-rata dari temp_min dan temp_max. Jika iya, nilai dapat direkonstruksi dari dua kolom terkait.
  * Kolom `temp_min` dan `temp_max` juga mengalami missing values (45 dan 22) → Hal ini bisa berdampak pada akurasi prediksi temperatur rata-rata (`temp_avg`).
  * Kolom `penyinaran_matahari` (24 missing values) → Penyinaran matahari berhubungan erat dengan cuaca dan penguapan, sehingga penting untuk mengisi data ini secara akurat.
  * Kolom `kec_angin`, `arah_angin`, dan `kec_angin_avg` memiliki jumlah missing values yang relatif kecil (17-18) → Bisa diatasi dengan metode imputasi sederhana tanpa banyak memengaruhi hasil analisis.

**Potensi Penyebab Missing Values**
  * Data tidak tercatat secara lengkap → Bisa terjadi karena kesalahan pencatatan atau alat pengukuran yang tidak berfungsi pada waktu tertentu.
  * Kondisi cuaca ekstrem → Pada kondisi tertentu (misalnya hujan deras atau kabut tebal), beberapa alat pengukuran mungkin gagal mencatat data dengan akurat.
  * Kesalahan dalam proses pengolahan data → Bisa jadi akibat penghapusan data duplikat atau konversi data yang menyebabkan beberapa nilai hilang.

  

In [ ]:
# @title 🔧 Handling Missing Values {"display-mode":"form"}
#Beberapa kolom diisi menggunakan interpolasi
kolom_diinterpolasi = ['temp_avg', 'kelembapan_avg', 'temp_min',
                        'penyinaran_matahari', 'temp_max', 'kec_angin',
                        'kec_angin_avg']

data_cuaca[kolom_diinterpolasi] = data_cuaca[kolom_diinterpolasi].interpolate(method='linear')

# Mengisi missing value curah_hujan dengan median
data_cuaca.loc[:, 'curah_hujan'] = data_cuaca['curah_hujan'].fillna(data_cuaca['curah_hujan'].median())

# Mengisi missing values pada arah_angin dengan nilai terbanyak (mode)
data_cuaca.loc[:, 'arah_angin'] = data_cuaca['arah_angin'].fillna(data_cuaca['arah_angin'].mode()[0])

data_cuaca.isna().sum()

,0
Tanggal,0
temp_min,0
temp_max,0
temp_avg,0
kelembapan_avg,0
curah_hujan,0
penyinaran_matahari,0
kec_angin,0
arah_angin,0
kec_angin_avg,0


**Analisis Handling Missing Values**
<br>Missing values ditangani dengan metode yang sesuai untuk masing-masing kolom: interpolasi untuk data kontinu, median untuk data dengan distribusi tidak normal, dan modus untuk data kategorik.
  * Kolom seperti `temp_avg`, `kelembapan_avg`, `temp_min`, `penyinaran_matahari`, `temp_max`, `kec_angin`, dan `kec_angin_avg` menggunakan interpolasi linear. Pendekatan ini cocok karena mempertahankan pola perubahan data seiring waktu tanpa memperkenalkan distorsi besar.
  * `Curah_hujan` diisi dengan median, menghindari pengaruh nilai ekstrem seperti `8888` yang sebelumnya dikonversi menjadi `NaN`. Penggunaan median lebih robust dibandingkan mean karena tidak terpengaruh oleh outlier yang dapat mendistorsi distribusi data.
  * Karena `arah_angin` adalah data kategorik yang merupakan arah angin yang diwakilkan oleh angka besar derajat arahnya, pendekatan terbaik adalah mengisi dengan nilai yang paling sering muncul (mode). Sehingga memastikan bahwa pengisian nilai tetap masuk akal berdasarkan distribusi asli data.

Pendekatan yang digunakan dalam penelitian ini sangat efektif dalam menangani *missing values* dengan mempertimbangkan karakteristik masing-masing fitur. *Interpolasi* memastikan kontinuitas data, *median* mengatasi outlier pada curah hujan, dan *modus* mempertahankan keakuratan data kategorik. Hasil akhirnya adalah dataset yang lebih bersih, lebih stabil, dan siap untuk dianalisis lebih lanjut! 🚀

In [ ]:
# @title 🏷️ Pelabelan Kelas {"display-mode":"form"}
def klasifikasi_kelas(curah_hujan):
    if curah_hujan < 0.5:
        return 1
    elif 0.5 <= curah_hujan <= 20:
        return 2
    elif 20 < curah_hujan <= 50:
        return 3
    elif 50 < curah_hujan <= 100:
        return 4
    elif 100 < curah_hujan <= 150:
        return 5
    else:
        return 6

# Menambahkan kolom kelas
data_cuaca['kelas'] = data_cuaca['curah_hujan'].apply(klasifikasi_kelas)

data_cuaca

,Tanggal,temp_min,temp_max,temp_avg,kelembapan_avg,curah_hujan,penyinaran_matahari,kec_angin,arah_angin,kec_angin_avg,kelas
0,2019-01-01,25.4,32.0,26.8,90.0,2.5,5.5,3.0,350.0,0.0,2
1,2019-01-02,24.8,32.0,27.0,89.0,32.6,1.6,4.0,340.0,1.0,3
2,2019-01-03,25.4,31.8,26.0,94.0,20.5,1.7,3.0,10.0,0.0,3
3,2019-01-04,24.0,31.0,26.6,88.0,4.0,0.0,6.0,10.0,1.0,2
4,2019-01-05,24.2,33.0,26.9,87.0,0.3,4.6,4.0,10.0,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...
2218,2025-01-27,25.0,30.0,27.4,86.0,1.5,6.2,5.0,40.0,1.0,2
2219,2025-01-28,25.4,31.6,27.8,87.0,0.0,5.2,6.0,360.0,1.0,1
2220,2025-01-29,25.2,31.4,27.1,87.0,0.0,3.5,5.0,360.0,1.0,1
2221,2025-01-30,25.2,31.6,26.5,93.0,14.2,6.2,5.0,110.0,1.0,2


**Analisis Pelabelan Kelas dalam Dataset**
<br>Penambahan kolom "kelas" dalam dataset yang sebelumnya hanya berisi variabel cuaca bertujuan untuk mengubah permasalahan dari regresi menjadi klasifikasi. Dengan adanya kolom ini, dataset kini dapat digunakan untuk memprediksi kategori curah hujan, bukan hanya nilainya secara langsung. Berikut adalah beberapa poin penting dari perubahan ini:

  * Sebelum: Dataset hanya berisi variabel numerik seperti temperatur, kelembapan, curah hujan, penyinaran matahari, dan kecepatan angin.
  * Sesudah: Dataset memiliki kolom "kelas", yang mengelompokkan curah hujan ke dalam empat kategori (1–4).
  * Dataset berubah menjadi dataset klasifikasi, sehingga metode pemodelan yang digunakan akan menyesuaikan ke algoritma klasifikasi seperti KNN, Random Forest, atau Decision Tree.
  * Metrik evaluasi juga berubah, dari MSE atau RMSE (untuk regresi) menjadi precision, recall, F1-score, dan confusion matrix untuk mengukur akurasi klasifikasi.

Sehingga memerlukan beberapa penyesuaian agar memastikan model tidak bias terhadap kelas mayoritas dan dapat melakukan klasifikasi dengan lebih akurat. 🚀


In [ ]:
# @title 🧾 Distribusi Kelas Target {"display-mode":"form"}

# Cek distribusi kelas target
distribusi = data_cuaca['kelas'].value_counts()

fig = px.bar(distribusi,
             x=distribusi.index,
             y=distribusi.values,
             title='Distribusi Kelas Target',
             color=distribusi.values,
             color_continuous_scale="purp"
             )

fig.data[0].hovertemplate = 'Kelas : %{x}<br>Jumlah : %{y}<extra></extra>'

fig.update_layout(
    title={'x': 0.5,
           'xanchor': 'center',
           'font_family': 'Arial Black',
           'font_size': 20,
           },
    xaxis=dict(
              title='Kelas',
              showline=True,
              linecolor='darkgrey'
          ),
    yaxis=dict(
              title='Jumlah',
              showline=True,
              linecolor='darkgrey'
          ),
    width=700,
    template='plotly_white',
    coloraxis_showscale=False,
    )

fig.add_annotation(xref='x domain', yref="y domain", x=0.5, y=1.13, font_size=14,
                   text="<i>Dataset yang digunakan cenderung tidak seimbang</i>", showarrow=False)

fig.show()

**Analisis Distribusi Data Berdasarkan Kelas**

Distribusi kelas dalam dataset menunjukkan ketidakseimbangan yang signifikan, yang dapat berdampak pada performa model klasifikasi.

**📊 Distribusi jumlah sampel per kelas**
  * Kelas 1 : 1.573 sampel (71.7%) → Mayoritas data berada di kelas ini.
  * Kelas 2 : 546 sampel (24.9%) → Jauh lebih sedikit dibanding kelas 1.
  * Kelas 3 : 92 sampel (4.2%) → Mulai sangat sedikit.
  * Kelas 4 : 12 sampel (0.5%) → Sangat minoritas, hampir tidak ada datanya.

🛑 Masalah utama : Dataset sangat tidak seimbang, terutama karena kelas 4 hanya memiliki 12 sampel dibanding kelas 1 yang memiliki 1573 sampel.

**Dampak pada Model Klasifikasi**
  * Model cenderung bias terhadap kelas mayoritas (Kelas 1).
  * Kelas dengan jumlah kecil (Kelas 3 & 4) mungkin tidak dikenali dengan baik, atau model akan mengabaikannya.
  * Precision dan recall untuk kelas 4 kemungkinan sangat rendah, karena model tidak memiliki cukup data untuk mempelajari pola kelas ini.
<br><br>


In [ ]:
# @title ℹ️ Informasi AKhir Dataset {"display-mode":"form"}
data_cuaca.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2223 entries, 0 to 2222
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Tanggal              2223 non-null   datetime64[ns]
 1   temp_min             2223 non-null   float64       
 2   temp_max             2223 non-null   float64       
 3   temp_avg             2223 non-null   float64       
 4   kelembapan_avg       2223 non-null   float64       
 5   curah_hujan          2223 non-null   float64       
 6   penyinaran_matahari  2223 non-null   float64       
 7   kec_angin            2223 non-null   float64       
 8   arah_angin           2223 non-null   float64       
 9   kec_angin_avg        2223 non-null   float64       
 10  kelas                2223 non-null   int64         
dtypes: datetime64[ns](1), float64(9), int64(1)
memory usage: 191.2 KB


**Analisis Output `info()` dari Dataset**

Berdasarkan hasil info(), dataset memiliki 2223 baris dan 11 kolom dengan berbagai tipe data. Berikut analisisnya:

**Kualitas Data**
  * Tidak ada missing values  
  Semua kolom memiliki 2223 non-null values, menunjukkan bahwa dataset lengkap tanpa nilai yang hilang (NaN).
  * Konsistensi tipe data
  <br>Kolom "Tanggal" bertipe datetime64, yang sudah sesuai untuk analisis berbasis waktu. 9 kolom numerik bertipe float64, yang cocok untuk analisis statistik dan machine learning. Kolom "kelas" bertipe int64, yang sudah sesuai karena merupakan target klasifikasi.

**Struktur Dataset**
  * Dataset terdiri dari 10 fitur dan 1 target (kelas)
  <br>Model akan menggunakan 10 fitur numerik untuk memprediksi kelas.
  * Ukuran dataset cukup besar (2223 baris)
  <br>Ini cukup baik untuk membangun model machine learning, tetapi tetap perlu diperiksa apakah semua kelas memiliki distribusi yang seimbang.
  * Jenis fitur yang digunakan
  <br> Fitur temperatur (temp_min, temp_max, temp_avg) → Memengaruhi pola cuaca dan curah hujan.
  <br> Fitur kelembapan (kelembapan_avg) → Berperan dalam pembentukan hujan.
  <br> Fitur curah hujan (curah_hujan) → Indikator utama dalam klasifikasi curah hujan.
  <br> Fitur angin (kec_angin, kec_angin_avg, arah_angin) → Dapat berpengaruh pada distribusi hujan dan kondisi cuaca ekstrem.
  <br> Fitur penyinaran matahari (penyinaran_matahari) → Bisa menjadi faktor dalam penguapan dan pembentukan awan.

  <br><br>

##🧩 **Skema 1: Klasifikasi Tanpa Normalisasi & SMOTE**

Pada skema ini, model klasifikasi diterapkan langsung pada dataset asli tanpa melakukan normalisasi fitur maupun penyeimbangan kelas menggunakan SMOTE. Hal ini bertujuan untuk melihat bagaimana performa model dalam kondisi mentah, di mana data masih mengandung skala yang berbeda-beda antar fitur dan distribusi kelas yang tidak seimbang.

In [ ]:
# @title ✂️ Split Data {"display-mode":"form"}

# Memisahkan fitur (X) dan target (y)
X = data_cuaca.drop(columns=['Tanggal', 'kelas'])  # Pastikan target sudah berbentuk label
y = data_cuaca['kelas']

# Split data menjadi train dan test set (70:30)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)


**Analisis Split Data**
  * `Tanggal` dihapus karena bukan fitur yang relevan dalam pemodelan klasifikasi. Sementara itu `Kelas` dipisahkan sebagai target (y), sementara fitur-fitur lain tetap berada dalam X.
  * Data dibagi menjadi 70% untuk pelatihan (train set) dan 30% untuk pengujian (test set). Rasio ini cukup umum dan seimbang, memastikan model memiliki cukup data untuk belajar sekaligus cukup data untuk validasi performanya.
  * Parameter `stratify=y` memastikan distribusi kelas di train set dan test set tetap sama dengan distribusi awal dataset. Hal ini dilakukan mengingat dataset memiliki ketidakseimbangan kelas (imbalance), di mana kelas 4 jauh lebih sedikit dibanding kelas lainnya.
  * Random_state=42 digunakan agar pembagian data tetap konsisten setiap kali kode dijalankan. Hal Ini memastikan eksperimen yang dilakukan bisa direplikasi tanpa adanya variasi acak yang memengaruhi hasil.
  * Setelah proses ini, dataset siap digunakan untuk membangun model klasifikasi, di mana `X_train` dan `y_train` akan digunakan untuk melatih model, sedangkan `X_test` dan `y_test` digunakan untuk menguji performanya.

In [ ]:
# @title ⚙️ Inisiasi dan Training Model {"display-mode":"form"}

# Inisialisasi model Random Forest
rf_model = RandomForestClassifier(random_state=42)

# Training model
rf_model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

**Analisis Inisiasi dan Training Model**

  * Model yang digunakan adalah *Random Forest Classifier*, yaitu algoritma berbasis pohon keputusan yang bekerja dengan membentuk banyak pohon (ensemble learning).
  * Penggunaan `random_state=42` memastikan bahwa proses pembuatan pohon dalam *Random Forest* bersifat deterministik (hasil yang sama setiap kali dijalankan).
  * Model dilatih menggunakan data latih `(X_train dan y_train)`. Random Forest akan membuat banyak pohon keputusan dengan cara mengambil sampel acak dari data dan memilih fitur secara acak untuk melakukan pembagian (splitting).
  * Setelah proses training ini selesai, model siap untuk diuji menggunakan `X_test` untuk mengevaluasi performa prediksinya berdasarkan metrik seperti `accuracy`, `precision`, `recall`,` F1-score`, dan `confusion matrix`.

In [ ]:
# @title 📝 Classification Report {"display-mode":"form"}

y_pred = rf_model.predict(X_test)

# === 1️⃣ Generate Classification Report ===
report = classification_report(y_test, y_pred, output_dict=True, zero_division=0)

# Konversi hasil classification report ke dalam bentuk DataFrame
df_report = pd.DataFrame(report).transpose().reset_index()
df_report.columns = ["Class", "Precision", "Recall", "F1-Score", "Support"]

# Mengatur format angka agar lebih rapi (hanya 2 angka di belakang koma)
df_report.iloc[:, 1:-1] = df_report.iloc[:, 1:-1].round(3)

# === 2️⃣ Menampilkan di Terminal dengan PrettyTable ===
table = PrettyTable()
table.field_names = df_report.columns.tolist()

for row in df_report.values:
    table.add_row(row)

print("\n📌 Classification Report dalam Bentuk Tabel:\n")
print(table)


# Evaluasi macro F1-score
macro_f1 = f1_score(y_test, y_pred, average='macro')
print(f"\n📌 Macro F1-score: {macro_f1:.4f}")


📌 Classification Report dalam Bentuk Tabel:

+--------------+-----------+--------+----------+--------------------+
|    Class     | Precision | Recall | F1-Score |      Support       |
+--------------+-----------+--------+----------+--------------------+
|      1       |    1.0    |  1.0   |   1.0    |       472.0        |
|      2       |    1.0    |  1.0   |   1.0    |       164.0        |
|      3       |   0.933   |  1.0   |  0.966   |        28.0        |
|      4       |    1.0    | 0.333  |   0.5    |        3.0         |
|   accuracy   |   0.997   | 0.997  |  0.997   | 0.9970014992503748 |
|  macro avg   |   0.983   | 0.833  |  0.866   |       667.0        |
| weighted avg |   0.997   | 0.997  |  0.996   |       667.0        |
+--------------+-----------+--------+----------+--------------------+

📌 Macro F1-score: 0.8664


**Analisis Classification Report**
<br> Laporan ini menunjukkan evaluasi kinerja model klasifikasi berdasarkan Precision, Recall, F1-Score, dan Support untuk setiap kelas.

**Kinerja Model Secara Keseluruhan**
  * Akurasi Sangat Tinggi (99.7%)
    <br> → Model bekerja dengan sangat baik dalam mengklasifikasikan data secara keseluruhan.
  * ✅ Weighted Avg F1-Score (0.996)
    <br> → Model hampir sempurna dalam prediksi secara proporsional terhadap jumlah data di setiap kelas.
  * ⚠️ Macro F1-Score (0.8664)
    <br> → Indikasi adanya ketidakseimbangan kinerja antar kelas, terutama kelas dengan jumlah sampel kecil.

**Analisis Per Kelas**
  * Kelas 1 & 2 → Model sempurna (Precision, Recall, dan F1 = 1.0).
  * Kelas 3 → Precision sedikit lebih rendah (0.933), tetapi Recall sempurna (1.0).
  * Kelas 4 → Precision sempurna (1.0), tetapi Recall sangat rendah (0.333), menyebabkan F1-Score hanya 0.5 → Model gagal mengidentifikasi kelas 4 dengan baik.

**Masalah pada Kelas 4**
  * Recall yang sangat rendah (0.333)
    <br> → Artinya hanya 33% dari kelas 4 yang berhasil dikenali oleh model, sementara sisanya salah diklasifikasikan sebagai kelas lain.
  * Support hanya 3 sampel
    <br> → Indikasi bahwa kelas ini sangat jarang muncul dalam dataset → Model kesulitan mengenali pola dari data yang sangat sedikit.

**Evaluasi Berdasarkan Metrik Makro vs. Weighted**
  * Weighted Avg (0.996)
    <br> → Model terlihat sangat baik secara keseluruhan karena mempertimbangkan jumlah sampel yang besar.
  * Macro Avg (0.866)
    <br> → Menunjukkan ada masalah ketidakseimbangan antar kelas, terutama karena performa buruk pada kelas 4.

**🚀 Kesimpulan**
<br>Model sangat baik secara keseluruhan (99.7% akurasi) tetapi masih memiliki kelemahan dalam mengenali kelas dengan jumlah sampel kecil (kelas 4).




In [ ]:
# @title 🧮 Confusion Matrix {"display-mode":"form"}

y_pred = rf_model.predict(X_test)


cm = confusion_matrix(y_test, y_pred)
labels = [str(i) for i in np.unique(y_test)]

fig = px.imshow(cm, color_continuous_scale='Purples',
                x=labels, y=labels, text_auto=True, labels={"color":"Jumlah"}, title="Confusion Matrix")

fig.update_traces(xgap=1, ygap=1)
fig.data[0].hovertemplate = 'Prediksi : %{x}<br>Aktual : %{y}<br>Jumlah : %{z}<extra></extra>'

fig.update_layout(
    title={'x': 0.5,
           'xanchor': 'center',
           'font_family': 'Arial Black',
           'font_size': 20,
           },
    xaxis=dict(
              title="Prediksi",
              showline=True,
              linecolor='darkgrey'
          ),
    yaxis=dict(
              title="Aktual",
              showline=True,
              linecolor='darkgrey'
          ),
    width=700,
    template='plotly_white',
    )

fig.add_annotation(xref='x domain', yref="y domain", x=0.5, y=1.13, font_size=14,
                   text="<i>Model berperforma baik hampir pada semua kelas</i>", showarrow=False)


fig.show()




**Analisis Kinerja Model**
  * Kelas 1 → 472 prediksi benar, 0 salah → Akurasi sempurna (100%).
  * Kelas 2 → 164 prediksi benar, 0 salah → Akurasi sempurna (100%).
  * Kelas 3 → 28 prediksi benar, 0 salah → Akurasi sempurna (100%).
  * Kelas 4 → Hanya 1 dari 3 prediksi yang benar, sementara 2 salah diklasifikasikan sebagai kelas 3

Terdapat Dua dari tiga sampel kelas 4 diklasifikasikan sebagai kelas 3. Hal ini dikarenakan kelas 3 dan 4 memiliki fitur yang mirip. Selain itu, keterbatasan jumlah data dari kelas 4 yang menyebabkan model tidak cukup belajar untuk membedakannya dari kelas 3.




In [ ]:
# @title ⚡ Feature Importance {"display-mode":"form"}

importances = rf_model.feature_importances_
feature_names = X.columns
feat_importance_df = pd.DataFrame({"Feature": feature_names, "Importance": importances})
feat_importance_df = feat_importance_df.sort_values(by="Importance", ascending=False)

# Visualisasi Feature Importance
fig = px.bar(feat_importance_df,
             x="Importance",
             y="Feature",
             orientation="h",
             title="Feature Importance",
             color='Importance',
             color_continuous_scale="purp"
             )

fig.data[0].texttemplate = '%{x:.3f}'
fig.data[0].textposition = 'outside'
fig.data[0].hovertemplate = 'Kepentingan : %{x:.3f}<br>Fitur : %{y}<extra></extra>'

fig.update_layout(
    title={'x': 0.5,
           'xanchor': 'center',
           'font_family': 'Arial Black',
           'font_size': 20,
           },
    xaxis=dict(
              range=[0, 0.9],
              title="Kepentingan",
              showline=True,
              linecolor='darkgrey'
          ),
    yaxis=dict(
              title="Fitur",
              showline=True,
              linecolor='darkgrey',
              categoryorder='total ascending'
          ),
    width=700,
    template='plotly_white',
    coloraxis_showscale=False,
    )

fig.add_annotation(xref='x domain', yref="y domain", x=0.4, y=1.13, font_size=14,
                   text="<i>Fitur curah_hujan berpengaruh besar pada model</i>", showarrow=False)

fig.show()

**Analisis Feature Importance**
<br>Hasil ini menunjukkan seberapa besar kontribusi setiap fitur dalam menentukan output model. Nilai feature importance yang lebih tinggi berarti fitur tersebut memiliki pengaruh yang lebih besar terhadap keputusan model.

**Fitur Paling Berpengaruh**
<br>🔥 Curah Hujan
  * Memiliki nilai feature importance tertinggi (0.754).
  * Sangat dominan dibanding fitur lain
  * Model sangat bergantung pada curah hujan untuk melakukan klasifikasi.
  * Wajar jika target yang diprediksi berhubungan dengan curah hujan harian.


**Fitur dengan Pengaruh Sedang**
<br>⚡ Kec. Angin Rata-rata, Penyinaran Matahari, Kelembapan Rata-rata
  * Berkontribusi cukup signifikan tetapi jauh lebih kecil dibanding curah hujan.
  * Mungkin masih memiliki hubungan dengan prediksi curah hujan, tetapi tidak sepenting curah hujan itu sendiri.


**Fitur dengan Pengaruh Rendah**
<br>💡 Temp. Min, Kec. Angin, Arah Angin, Temp. Max, Temp. Avg
  * Meskipun termasuk dalam model, pengaruhnya sangat kecil.
  * Temperatur dan arah angin tampaknya tidak terlalu berkontribusi terhadap prediksi curah hujan.
  * Bisa dipertimbangkan untuk dibuang (feature selection) jika ingin menyederhanakan model tanpa mengurangi akurasi terlalu banyak.


**🚀 Kesimpulan akhir**
 <br>Model sangat mengandalkan curah hujan sebagai fitur utama, sementara fitur lain hanya memiliki kontribusi kecil hingga sangat kecil.

 <br><br>

##🧩 **Skema 2: Klasifikasi Dengan SMOTE**

Setelah melakukan klasifikasi tanpa resampling pada dataset awal, hasil evaluasi menunjukkan adanya ketidakseimbangan kelas yang cukup signifikan. Hal ini berdampak pada performa model, terutama dalam mengklasifikasikan kelas minoritas. Untuk mengatasi ketimpangan ini, pada skema kedua, Synthetic Minority Over-sampling Technique (SMOTE) diterapkan guna menyeimbangkan distribusi data sebelum proses pelatihan model dilakukan.

In [ ]:
# @title ✂️ Split Data {"display-mode":"form"}

# Memisahkan fitur (X) dan target (y)
X = data_cuaca.drop(columns=['Tanggal', 'kelas'])  # Pastikan target sudah berbentuk label
y = data_cuaca['kelas']

# Split data menjadi train dan test set (70:30)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Cek distribusi kelas sebelum resampling
distribution_before = y_train.value_counts().reset_index()
distribution_before.columns = ['Kelas', 'Jumlah']
distribution_before['Tipe'] = 'Sebelum Resampling'

distribution_before

,Kelas,Jumlah,Tipe
0,1,1101,Sebelum Resampling
1,2,382,Sebelum Resampling
2,3,64,Sebelum Resampling
3,4,9,Sebelum Resampling


**Analisis Distribusi Data Sebelum Resampling**
<br> Tabel tersebut menunjukkan persebaran data pada setiap kelas sebelum dilakukannya resampling. Berikut ini adalah penjelasan utuh dari analisis distribusi tersebut.
  * Kelas 1 memiliki jumlah sampel paling banyak (1101), sedangkan kelas 4 memiliki jumlah sampel sangat sedikit (9). Disisi lain, kelas 2 dan 3 juga memiliki jumlah sampel yang jauh lebih sedikit dibandingkan kelas 1, yaitu 382 dan 64.
  * Model cenderung lebih akurat dalam memprediksi kelas mayoritas (1) karena jumlah datanya jauh lebih besar. Sedangkan kelas minoritas (terutama kelas 3 dan 4) bisa mengalami underfitting, di mana model kesulitan mempelajari pola yang cukup dari jumlah sampel yang kecil.

Distribusi data sebelum resampling menunjukkan ketidakseimbangan yang cukup ekstrem antara kelas mayoritas dan minoritas. Hal ini berisiko menyebabkan model bias terhadap kelas mayoritas dan menurunkan performa prediksi pada kelas minoritas. Oleh karena itu, SMOTE menjadi solusi yang tepat untuk memperbaiki distribusi ini sehingga model dapat belajar dengan lebih baik dan menghasilkan prediksi yang lebih akurat.

In [ ]:
# @title 🧪 Oversampling Data {"display-mode":"form"}

# Oversampling dengan SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

distribution_after = pd.Series(y_train_resampled).value_counts().reset_index()
distribution_after.columns = ['Kelas', 'Jumlah']
distribution_after['Tipe'] = 'Setelah Resampling'

distribution_after

,Kelas,Jumlah,Tipe
0,1,1101,Setelah Resampling
1,2,1101,Setelah Resampling
2,3,1101,Setelah Resampling
3,4,1101,Setelah Resampling


**Analisis Distribusi Data Setelah Resampling Menggunakan SMOTE**
<br>Setelah diterapkan Synthetic Minority Over-sampling Technique (SMOTE), distribusi data antar kelas menjadi seimbang, dengan masing-masing kelas memiliki 1.101 sampel.
  * SMOTE berhasil mengatasi ketidakseimbangan kelas. Hal ini terlihat dari sebelum resampling, kelas 3 dan 4 sangat jarang muncul, sedangkan setelahnya jumlahnya menjadi seimbang dengan kelas lainnya.
  * Mencegah bias model terhadap kelas mayoritas dimana model tidak lagi cenderung memprediksi kelas yang paling dominan karena semua kelas memiliki jumlah sampel yang sama.
  * Meningkatkan performa model untuk kelas minoritas yang mana sebelum SMOTE, model mungkin kesulitan mengenali kelas 3 dan 4 karena jumlah datanya sangat sedikit.

SMOTE berhasil menyeimbangkan jumlah sampel di setiap kelas, yang sangat penting dalam meningkatkan akurasi model untuk kelas minoritas. Namun, perlu hati-hati terhadap overfitting dan distorsi distribusi data. Validasi lebih lanjut dengan data uji yang sesungguhnya sangat disarankan agar model tetap generalizable dan tidak hanya "menghafal" pola dari data sintetis. 🚀

In [ ]:
# @title 📤 Distribusi Kelas {"display-mode":"form"}

# Gabungkan kedua distribusi untuk visualisasi
distribution_combined = pd.concat([distribution_before, distribution_after])

# Visualisasi distribusi kelas sebelum dan sesudah resampling
fig = px.bar(distribution_combined,
             x='Kelas',
             y='Jumlah',
             color='Tipe',
             barmode='group',
             title='Distribusi Kelas Sebelum dan Sesudah Resampling',
             color_discrete_sequence=['pink', 'mediumpurple'])

fig.data[0].hovertemplate = 'Kelas : %{x}<br>Jumlah : %{y}<extra></extra>'
fig.data[1].hovertemplate = 'Kelas : %{x}<br>Jumlah : %{y}<extra></extra>'

fig.update_layout(
    title={'x': 0.5,
           'xanchor': 'center',
           'font_family': 'Arial Black',
           'font_size': 20,
           },
    xaxis=dict(
              title="Kelas",
              showline=True,
              linecolor='darkgrey'
          ),
    yaxis=dict(
              title="Jumlah",
              showline=True,
              linecolor='darkgrey',
          ),
    legend=dict(
              title="",
              orientation="h",
              yanchor="bottom",
              y=-0.25,
              xanchor="center",
              x=0.5
              ),
    width=700,
    template='plotly_white',
    )

fig.add_annotation(xref='x domain', yref="y domain", x=0.5, y=1.14, font_size=14,
                   text="<i>Distribusi kelas menjadi rata sesudah resampling</i>", showarrow=False)

fig.show()

**Analisis Perbandingan Distribusi Data Sebelum & Sesudah Resampling dengan SMOTE**

  * Sebelum resampling, distribusi kelas sangat tidak seimbang dengan kelas mayoritas (1: 1101 sampel) dan kelas minoritas (4: hanya 9 sampel). Kelas 2 dan 3 juga memiliki jumlah sampel yang jauh lebih sedikit dibanding kelas 1, yakni 382 dan 64. Dengan distribusi seperti ini, model berisiko menjadi bias terhadap kelas mayoritas, sehingga sulit mengenali pola dari kelas yang memiliki data lebih sedikit.
  * Setelah menggunakan SMOTE (Synthetic Minority Over-sampling Technique), jumlah sampel tiap kelas diseimbangkan menjadi 1101. Teknik ini menambahkan data sintetis untuk kelas 2, 3, dan 4, sehingga distribusi data menjadi seragam di antara semua kelas.

Dengan menerapkan SMOTE, distribusi data menjadi lebih seimbang, yang sangat penting untuk meningkatkan performa model dalam mengklasifikasikan semua kelas secara adil. Meskipun ada risiko overfitting, pendekatan ini tetap lebih baik dibanding membiarkan model hanya belajar dari data yang sangat tidak seimbang. 🚀

In [ ]:
# @title ⚙️ Inisiasi dan Training Model {"display-mode":"form"}

# Inisialisasi model Random Forest
rf_resampling = RandomForestClassifier(random_state=42)

# Training model dengan data resampled
rf_resampling.fit(X_train_resampled, y_train_resampled)

RandomForestClassifier(random_state=42)

**Analisis Inisiasi dan Training Model Klasifikasi Dengan SMOTE**

  * Model yang digunakan adalah *Random Forest Classifier*, yaitu algoritma berbasis pohon keputusan yang bekerja dengan membentuk banyak pohon (ensemble learning).
  * Penggunaan `random_state=42` memastikan bahwa proses pembuatan pohon dalam *Random Forest* bersifat deterministik (hasil yang sama setiap kali dijalankan).
  * Model dilatih menggunakan data latih `(X_train_resampled dan y_train_resampled)`. Data latih yang digunakan telah diresampling sebelumnya, sehingga diharapkan membuat model dapat bekerja lebih optimal. Random Forest akan membuat banyak pohon keputusan dengan cara mengambil sampel acak dari data dan memilih fitur secara acak untuk melakukan pembagian (splitting).
  * Setelah proses training ini selesai, model siap untuk diuji menggunakan `X_test` untuk mengevaluasi performa prediksinya berdasarkan metrik seperti `accuracy`, `precision`, `recall`,` F1-score`, dan `confusion matrix`.

In [ ]:
# @title 📝 Classification Report {"display-mode":"form"}

y_pred_resampling = rf_resampling.predict(X_test)

# === 1️⃣ Generate Classification Report ===
report_resampling = classification_report(y_test, y_pred_resampling, output_dict=True, zero_division=0)

# Konversi hasil classification report_resampling ke dalam bentuk DataFrame
df_report_resampling = pd.DataFrame(report_resampling).transpose().reset_index()
df_report_resampling.columns = ["Class", "Precision", "Recall", "F1-Score", "Support"]

# Mengatur format angka agar lebih rapi (hanya 2 angka di belakang koma)
df_report_resampling.iloc[:, 1:-1] = df_report_resampling.iloc[:, 1:-1].round(3)

# === 2️⃣ Menampilkan di Terminal dengan PrettyTable ===
table_resampling = PrettyTable()
table_resampling.field_names = df_report_resampling.columns.tolist()

for row in df_report_resampling.values:
    table_resampling.add_row(row)

print("\n📌 Classification Report Klasifikasi Dengan SMOTE:\n")
print(table_resampling)


# Evaluasi macro F1-score
macro_f1_norm = f1_score(y_test, y_pred_resampling, average='macro')
print(f"\n📌 Macro F1-score: {macro_f1_norm:.4f}")


📌 Classification Report Klasifikasi Dengan SMOTE:

+--------------+-----------+--------+----------+---------+
|    Class     | Precision | Recall | F1-Score | Support |
+--------------+-----------+--------+----------+---------+
|      1       |    1.0    |  1.0   |   1.0    |  472.0  |
|      2       |    1.0    |  1.0   |   1.0    |  164.0  |
|      3       |    1.0    |  1.0   |   1.0    |   28.0  |
|      4       |    1.0    |  1.0   |   1.0    |   3.0   |
|   accuracy   |    1.0    |  1.0   |   1.0    |   1.0   |
|  macro avg   |    1.0    |  1.0   |   1.0    |  667.0  |
| weighted avg |    1.0    |  1.0   |   1.0    |  667.0  |
+--------------+-----------+--------+----------+---------+

📌 Macro F1-score: 1.0000


**Analisis Classification Report Klasifikasi Dengan SMOTE**

  * Semua metrik (Precision, Recall, dan F1-Score) untuk setiap kelas bernilai 1.0, yang artinya model memprediksi semua sampel dengan sempurna. Akurasi 100% seperti ini sangat jarang terjadi dalam masalah klasifikasi dunia nyata, sehingga ada kemungkinan model mengalami overfitting terhadap data latih.
  * Kelas 1 (Mayoritas): F1-score 1.0, artinya model mampu mengidentifikasi semua sampel kelas 1 dengan sempurna. Kelas 2, 3, dan 4 (Minoritas): Juga memiliki F1-score 1.0, yang menunjukkan tidak ada kesalahan prediksi sama sekali. Macro F1-score = 1.0000, menandakan model tidak memberikan perlakuan berbeda pada kelas mayoritas dan minoritas setelah SMOTE diterapkan.
  

In [ ]:
# @title 🧮 Confusion Matrix {"display-mode":"form"}

cm = confusion_matrix(y_test, y_pred_resampling)
labels = [str(i) for i in np.unique(y_test)]

fig = px.imshow(cm, color_continuous_scale='Purples',
                x=labels, y=labels, text_auto=True, labels={"color":"Jumlah"}, title="Confusion Matrix")

fig.update_traces(xgap=1, ygap=1)
fig.data[0].hovertemplate = 'Prediksi : %{x}<br>Aktual : %{y}<br>Jumlah : %{z}<extra></extra>'

fig.update_layout(
    title={'x': 0.5,
           'xanchor': 'center',
           'font_family': 'Arial Black',
           'font_size': 20,
           },
    xaxis=dict(
              title="Prediksi",
              showline=True,
              linecolor='darkgrey'
          ),
    yaxis=dict(
              title="Aktual",
              showline=True,
              linecolor='darkgrey'
          ),
    width=700,
    template='plotly_white',
    )

fig.add_annotation(xref='x domain', yref="y domain", x=0.5, y=1.13, font_size=14,
                   text="<i>Model yang dihasilkan sangat akurat</i>", showarrow=False)


fig.show()


**Analisis Confusion Matrix Klasifikasi Dengan SMOTE**

Confusion matrix ini menunjukkan tidak ada kesalahan klasifikasi sama sekali, karena semua prediksi berada di diagonal utama. Artinya, Model mampu mengenali setiap kelas tanpa kesalahan. Precision, Recall, dan F1-score sempurna (sejalan dengan classification report sebelumnya). Kondisi ini kemungkinan disebabkan oleh beberapa hal berikut ini
  *  Overfitting akibat SMOTE → Model mungkin terlalu terlatih pada data sintetis yang mirip dengan data asli, sehingga tidak melakukan kesalahan pada data uji.
  * Data uji tidak cukup menantang → Jika data uji memiliki pola yang terlalu mirip dengan data latih, model bisa mengenali semuanya tanpa kesalahan.
  * Model terlalu kompleks → Random Forest dengan banyak estimators atau tanpa batasan max_depth bisa menyebabkan model terlalu "menghafal" pola.

Confusion matrix ini menunjukkan tidak ada kesalahan klasifikasi sama sekali, yang sangat tidak biasa dalam skenario dunia nyata. Ada indikasi kuat bahwa model mengalami overfitting, kemungkinan besar akibat SMOTE atau data uji yang tidak cukup kompleks. Validasi lebih lanjut sangat diperlukan untuk memastikan model benar-benar dapat menggeneralisasi data baru dengan baik. 🚀

In [ ]:
# @title ⚡ Feature Importance {"display-mode":"form"}

importances = rf_resampling.feature_importances_
feature_names = X.columns
feat_importance_df = pd.DataFrame({"Feature": feature_names, "Importance": importances})
feat_importance_df = feat_importance_df.sort_values(by="Importance", ascending=False)

# Visualisasi Feature Importance
fig = px.bar(feat_importance_df,
             x="Importance",
             y="Feature",
             orientation="h",
             title="Feature Importance",
             color='Importance',
             color_continuous_scale="purp"
             )

fig.data[0].texttemplate = '%{x:.3f}'
fig.data[0].textposition = 'outside'
fig.data[0].hovertemplate = 'Kepentingan : %{x:.3f}<br>Fitur : %{y}<extra></extra>'

fig.update_layout(
    title={'x': 0.5,
           'xanchor': 'center',
           'font_family': 'Arial Black',
           'font_size': 20,
           },
    xaxis=dict(
              range=[0, 0.9],
              title="Kepentingan",
              showline=True,
              linecolor='darkgrey'
          ),
    yaxis=dict(
              title="Fitur",
              showline=True,
              linecolor='darkgrey',
              categoryorder='total ascending'
          ),
    width=700,
    template='plotly_white',
    coloraxis_showscale=False,
    )

fig.add_annotation(xref='x domain', yref="y domain", x=0.4, y=1.13, font_size=14,
                   text="<i>Fitur curah_hujan berpengaruh besar pada model</i>", showarrow=False)

fig.show()

**Analisis Feature Importance**
<br>Hasil ini menunjukkan seberapa besar kontribusi setiap fitur dalam menentukan output model. Nilai feature importance yang lebih tinggi berarti fitur tersebut memiliki pengaruh yang lebih besar terhadap keputusan model.

  * `curah_hujan` memiliki nilai feature importance tertinggi (0.716) yang membuatnya sangat dominan dibanding fitur lain. Sehingga model sangat bergantung pada curah hujan untuk melakukan klasifikasi.
  * `kec_angin_avg`, `penyinaran_matahari`, dan `kelembapan_avg` berkontribusi cukup signifikan tetapi jauh lebih kecil dibanding `curah_hujan`.
  * `temp_min`, `kec_angin`, `arah_angin`, `temp_max`, serta `temp_avg` meskipun termasuk dalam model, pengaruhnya sangat kecil.

Model sangat mengandalkan `curah_hujan` sebagai fitur utama, sementara fitur lain hanya memiliki kontribusi kecil hingga sangat kecil.


##🧩 **Skema 3: Klasifikasi Dengan Fitur Pilihan**

Setelah menerapkan metode klasifikasi pada dataset yang telah diseimbangkan dengan SMOTE, langkah selanjutnya adalah mengeksplorasi kemungkinan peningkatan performa model dengan melakukan seleksi fitur. Tidak semua fitur dalam dataset memiliki kontribusi yang signifikan terhadap performa model, dan beberapa di antaranya mungkin justru menambah kompleksitas tanpa memberikan informasi yang substansial. Oleh karena itu, pada skema ini dilakukan proses pemilihan fitur guna mengoptimalkan kinerja model Random Forest.

In [ ]:
# @title ✂️ Split Data {"display-mode":"form"}

# Gunakan hanya fitur yang paling berpengaruh
selected_features = ['curah_hujan']  # Fitur yang paling berpengaruh berdasarkan percobaan sebelumnya
X = data_cuaca[selected_features]
y = data_cuaca['kelas']

# Split data (70% training, 30% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

**Analisis Split Data Klasifikasi Dengan Fitur Pilihan**
  * `curah_hujan` dipilih karena fitur ini adalah yang baling berpengaruh berdasarkan percobaan sebelumnya. Sementara itu `Kelas` dipisahkan sebagai target (y), sementara fitur-fitur lain tetap berada dalam X.
  * Data dibagi menjadi 70% untuk pelatihan (train set) dan 30% untuk pengujian (test set). Rasio ini cukup umum dan seimbang, memastikan model memiliki cukup data untuk belajar sekaligus cukup data untuk validasi performanya.
  * Parameter `stratify=y` memastikan distribusi kelas di train set dan test set tetap sama dengan distribusi awal dataset. Hal ini dilakukan mengingat dataset memiliki ketidakseimbangan kelas (*imbalance*), di mana kelas 4 jauh lebih sedikit dibanding kelas lainnya.
  * `Random_state=42` digunakan agar pembagian data tetap konsisten setiap kali kode dijalankan. Hal Ini memastikan eksperimen yang dilakukan bisa direplikasi tanpa adanya variasi acak yang memengaruhi hasil.
  * Setelah proses ini, dataset siap digunakan untuk membangun model klasifikasi, di mana `X_train` dan `y_train` akan digunakan untuk melatih model, sedangkan `X_test` dan `y_test` digunakan untuk menguji performanya.

In [ ]:
# @title ⚙️ Inisiasi dan Training Model {"display-mode":"form"}

# Inisialisasi model Random Forest
rf_selected = RandomForestClassifier(random_state=42)

# Training model dengan data resampled
rf_selected.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

**Analisis Inisiasi dan Training Model Klasifikasi Dengan Fitur Pilihan**

  * Model yang digunakan adalah *Random Forest Classifier*, yaitu algoritma berbasis pohon keputusan yang bekerja dengan membentuk banyak pohon (ensemble learning).
  * Penggunaan `random_state=42` memastikan bahwa proses pembuatan pohon dalam *Random Forest* bersifat deterministik (hasil yang sama setiap kali dijalankan).
  * Model dilatih menggunakan data latih `(X_train dan y_train)`. Random Forest akan membuat banyak pohon keputusan dengan cara mengambil sampel acak dari data dan memilih fitur secara acak untuk melakukan pembagian (splitting).
  * Setelah proses training ini selesai, model siap untuk diuji menggunakan `X_test` untuk mengevaluasi performa prediksinya berdasarkan metrik seperti `accuracy`, `precision`, `recall`,` F1-score`, dan `confusion matrix`.

In [ ]:
# @title 📝 Classification Report {"display-mode":"form"}

y_pred_selected = rf_selected.predict(X_test)

# === 1️⃣ Generate Classification Report ===
report_selected = classification_report(y_test, y_pred_selected, output_dict=True, zero_division=0)

# Konversi hasil classification report_selected ke dalam bentuk DataFrame
df_report_selected = pd.DataFrame(report_selected).transpose().reset_index()
df_report_selected.columns = ["Class", "Precision", "Recall", "F1-Score", "Support"]

# Mengatur format angka agar lebih rapi (hanya 2 angka di belakang koma)
df_report_selected.iloc[:, 1:-1] = df_report_selected.iloc[:, 1:-1].round(3)

# === 2️⃣ Menampilkan di Terminal dengan PrettyTable ===
table_selected = PrettyTable()
table_selected.field_names = df_report_selected.columns.tolist()

for row in df_report_selected.values:
    table_selected.add_row(row)

print("\n📌 Classification Report Klasifikasi Dengan Fitur Pilihan:\n")
print(table_selected)


# Evaluasi macro F1-score
macro_f1_norm = f1_score(y_test, y_pred_selected, average='macro')
print(f"\n📌 Macro F1-score: {macro_f1_norm:.4f}")


📌 Classification Report Klasifikasi Dengan Fitur Pilihan:

+--------------+-----------+--------+----------+---------+
|    Class     | Precision | Recall | F1-Score | Support |
+--------------+-----------+--------+----------+---------+
|      1       |    1.0    |  1.0   |   1.0    |  472.0  |
|      2       |    1.0    |  1.0   |   1.0    |  164.0  |
|      3       |    1.0    |  1.0   |   1.0    |   28.0  |
|      4       |    1.0    |  1.0   |   1.0    |   3.0   |
|   accuracy   |    1.0    |  1.0   |   1.0    |   1.0   |
|  macro avg   |    1.0    |  1.0   |   1.0    |  667.0  |
| weighted avg |    1.0    |  1.0   |   1.0    |  667.0  |
+--------------+-----------+--------+----------+---------+

📌 Macro F1-score: 1.0000


**Analisis Classification Report Klasifikasi Dengan Fitur Pilihan**

  * Semua metrik (Precision, Recall, dan F1-Score) untuk setiap kelas bernilai 1.0, yang artinya model memprediksi semua sampel dengan sempurna. Akurasi 100% seperti ini sangat jarang terjadi dalam masalah klasifikasi dunia nyata, sehingga ada kemungkinan model mengalami overfitting terhadap data latih.
  * Kelas 1 (Mayoritas): F1-score 1.0, artinya model mampu mengidentifikasi semua sampel kelas 1 dengan sempurna. Kelas 2, 3, dan 4 (Minoritas): Juga memiliki F1-score 1.0, yang menunjukkan tidak ada kesalahan prediksi sama sekali. Macro F1-score = 1.0000, menandakan model tidak memberikan perlakuan berbeda pada kelas mayoritas dan minoritas setelah SMOTE diterapkan.

In [ ]:
# @title 🧮 Confusion Matrix {"display-mode":"form"}

cm = confusion_matrix(y_test, y_pred_selected)
labels = [str(i) for i in np.unique(y_test)]

fig = px.imshow(cm, color_continuous_scale='Purples',
                x=labels, y=labels, text_auto=True, labels={"color":"Jumlah"}, title="Confusion Matrix")

fig.update_traces(xgap=1, ygap=1)
fig.data[0].hovertemplate = 'Prediksi : %{x}<br>Aktual : %{y}<br>Jumlah : %{z}<extra></extra>'

fig.update_layout(
    title={'x': 0.5,
           'xanchor': 'center',
           'font_family': 'Arial Black',
           'font_size': 20,
           },
    xaxis=dict(
              title="Prediksi",
              showline=True,
              linecolor='darkgrey'
          ),
    yaxis=dict(
              title="Aktual",
              showline=True,
              linecolor='darkgrey'
          ),
    width=700,
    template='plotly_white',
    )

fig.add_annotation(xref='x domain', yref="y domain", x=0.5, y=1.13, font_size=14,
                   text="<i>Model yang dihasilkan sanggat akurat</i>", showarrow=False)


fig.show()


**Analisis Confusion Matrix Klasifikasi Dengan Fitur Pilihan**

Confusion matrix ini menunjukkan tidak ada kesalahan klasifikasi sama sekali, karena semua prediksi berada di diagonal utama. Artinya, Model mampu mengenali setiap kelas tanpa kesalahan. Precision, Recall, dan F1-score sempurna (sejalan dengan classification report sebelumnya). Kondisi ini kemungkinan disebabkan oleh beberapa hal berikut ini
  *  Model mungkin terlalu terlatih pada data sintetis yang mirip dengan data asli, sehingga tidak melakukan kesalahan pada data uji.
  * Jika data uji memiliki pola yang terlalu mirip dengan data latih, model bisa mengenali semuanya tanpa kesalahan.
  * Random Forest dengan banyak estimators atau tanpa batasan max_depth bisa menyebabkan model terlalu "menghafal" pola.

Confusion matrix ini menunjukkan tidak ada kesalahan klasifikasi sama sekali, yang sangat tidak biasa dalam skenario dunia nyata. Ada indikasi kuat bahwa model mengalami overfitting, kemungkinan besar akibat SMOTE atau data uji yang tidak cukup kompleks.

##🏁 **Penutup**

###💡 **Kesimpulan**

Dalam penelitian ini, kami telah mengeksplorasi berbagai pendekatan dalam klasifikasi curah hujan harian menggunakan metode Random Forest. Dengan tiga percobaan berbeda—klasifikasi langsung, SMOTE untuk menangani ketidakseimbangan data, dan seleksi fitur utama—beberapa temuan utama dapat disimpulkan:

  * **Pengaruh Ketidakseimbangan Data** ⚖️ <br>
    Distribusi kelas yang tidak merata berdampak pada akurasi model, terutama dalam mengklasifikasikan kelas dengan jumlah data yang sangat sedikit.

  * **Peningkatan Akurasi dengan SMOTE** 🔄 <br>
SMOTE berhasil meningkatkan akurasi dan performa model dengan menyeimbangkan jumlah data di setiap kelas, tetapi berpotensi menyebabkan overfitting.

  * **Kekuatan Seleksi Fitur** 🎯 <br>
Hanya dengan menggunakan fitur `curah_hujan`, model tetap mampu mencapai akurasi tinggi, menunjukkan bahwa fitur ini memiliki kontribusi paling besar dalam menentukan kelas curah hujan.



###💡 **Implikasi Penelitian**
Hasil dari penelitian ini memiliki beberapa implikasi penting:

  * **Bagi Meteorologi dan Klimatologi** 🌦️
  <br>Model ini dapat digunakan sebagai dasar awal dalam prakiraan cuaca berbasis pembelajaran mesin, terutama dalam mengklasifikasikan tingkat curah hujan harian.

  * **Bagi Sektor Pertanian dan Perencanaan Sumber Daya Air** 🏞️
  <br>Informasi yang dihasilkan dapat membantu petani dan pengelola sumber daya air dalam mengantisipasi pola curah hujan berdasarkan faktor iklim lainnya.

  * **Dalam Pengembangan Model Klasifikasi Cuaca** ⚛︎
  <br>Teknik *Resampling* yang digunakan dalam penelitian ini dapat diaplikasikan dalam model klasifikasi cuaca lainnya untuk meningkatkan akurasi prediksi.

###💡 **Saran**
Hasil penelitian ini dapat memberikan wawasan bagi para peneliti dan praktisi dalam analisis data cuaca dan machine learning. Dengan memahami dampak ketidakseimbangan data dan pentingnya seleksi fitur, model klasifikasi dapat dioptimalkan untuk meningkatkan akurasi prediksi.

Untuk pengembangan lebih lanjut, beberapa arah penelitian yang dapat dilakukan antara lain: <br>
  *  Eksperimen dengan model lain seperti XGBoost atau Neural Networks untuk membandingkan performa. <br>
  *  Validasi lebih mendalam menggunakan cross-validation untuk memastikan model tidak overfit. <br>
  * Eksplorasi lebih lanjut optimasi parameter pada Random Forest untuk meningkatkan akurasi prediksi.
  *  Analisis lebih lanjut terkait dampak cuaca terhadap tren dan pola perubahan curah hujan. <br>

Semoga penelitian ini dapat memberikan manfaat dan menjadi referensi bagi mereka yang tertarik dalam bidang data science, analisis cuaca, dan machine learning! 🚀🌧️📊